In [3]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [4]:
word = "yooks"
sample_text = [
"yooks are not zooks.",
"yooks know how to cook.",
"yooks are from yooktown.",
"yooks are not from zooktown",
"yooks like to cook.",
"yooks like to read books.",
"yooks like to read magazines.",
"there are no yooks in zooktown."
]

In [5]:
pattern = re.compile("yooks")

In [6]:
iters = [pattern.finditer(text) for text in sample_text]

In [7]:
tokenized_text = [word_tokenize(text) for text in  sample_text]

In [8]:
tokenized_text

[['yooks', 'are', 'not', 'zooks', '.'],
 ['yooks', 'know', 'how', 'to', 'cook', '.'],
 ['yooks', 'are', 'from', 'yooktown', '.'],
 ['yooks', 'are', 'not', 'from', 'zooktown'],
 ['yooks', 'like', 'to', 'cook', '.'],
 ['yooks', 'like', 'to', 'read', 'books', '.'],
 ['yooks', 'like', 'to', 'read', 'magazines', '.'],
 ['there', 'are', 'no', 'yooks', 'in', 'zooktown', '.']]

In [9]:
word_indices = [text.index("yooks") for text in tokenized_text]

In [10]:
word_indices

[0, 0, 0, 0, 0, 0, 0, 3]

In [84]:
raw_tree = {"name": word}
raw_tree["raw_children"] = [text[i+1:] for i,text in zip(word_indices,tokenized_text)]

In [85]:
raw_tree

{'name': 'yooks',
 'raw_children': [['are', 'not', 'zooks', '.'],
  ['know', 'how', 'to', 'cook', '.'],
  ['are', 'from', 'yooktown', '.'],
  ['are', 'not', 'from', 'zooktown'],
  ['like', 'to', 'cook', '.'],
  ['like', 'to', 'read', 'books', '.'],
  ['like', 'to', 'read', 'magazines', '.'],
  ['in', 'zooktown', '.']]}

In [86]:
def process_child(child):
  final = {"name": child[0]}
  if len(child)==1:
    final["size"]=1
  else:
    final["children"] = [process_child(child[1:])]
  return final
raw_tree["children"] = [process_child(child) for child in raw_tree["raw_children"]]

In [87]:
#print(raw_tree)

In [88]:
def combine_children(children, depth=0):
  new_children = {}
  for child in children:
    if child["name"] in new_children:
      new_children[child["name"]].get("children")
      if child.get("children") and new_children[child["name"]].get("children"):
        new_children[child["name"]]["children"] = combine_children(list(child["children"])+ list(new_children[child["name"]]["children"]), depth=depth+1)
      elif child.get("children"):
        new_children[child["name"]]["children"] = child["children"]
      new_children[child["name"]]["size"] = new_children[child["name"]].get("size",0) + child.get("size",0)
    else:
      new_children[child["name"]] = child
  return list(new_children.values())

In [89]:
raw_tree["children"] = combine_children(raw_tree["children"])

In [90]:
raw_tree["children"]

[{'children': [{'children': [{'children': [{'name': '.', 'size': 1}],
      'name': 'zooks'},
     {'children': [{'name': 'zooktown', 'size': 1}], 'name': 'from'}],
    'name': 'not',
    'size': 0},
   {'children': [{'children': [{'name': '.', 'size': 1}], 'name': 'yooktown'}],
    'name': 'from'}],
  'name': 'are',
  'size': 0},
 {'children': [{'children': [{'children': [{'children': [{'name': '.',
          'size': 1}],
        'name': 'cook'}],
      'name': 'to'}],
    'name': 'how'}],
  'name': 'know'},
 {'children': [{'children': [{'children': [{'name': '.', 'size': 1}],
      'name': 'cook'},
     {'children': [{'children': [{'name': '.', 'size': 1}],
        'name': 'magazines'},
       {'children': [{'name': '.', 'size': 1}], 'name': 'books'}],
      'name': 'read',
      'size': 0}],
    'name': 'to',
    'size': 0}],
  'name': 'like',
  'size': 0},
 {'children': [{'children': [{'name': '.', 'size': 1}], 'name': 'zooktown'}],
  'name': 'in'}]

In [95]:
new_tree = raw_tree.copy()
new_tree["raw_children"] = None

In [96]:
import json
with open('tree.json', 'w') as outfile:
    json.dump(new_tree, outfile)